In [1]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import time
import datetime

import geopandas as gpd

pd.options.mode.chained_assignment = None

In [2]:
# ====== Connection ======
# Connecting to PostgreSQL by providing a sqlachemy engine
# Local
local_username= 'ds4a_final'
local_password='qwerty.123'
local_host = 'localhost'
local_port='5432'
local_database = 'ds4a_finalproject'

local_engine = create_engine(f'postgresql://{local_username}:{local_password}@{local_host}:{local_port}/{local_database}',echo=False)

# Remote
remote_username= 'final_project'
remote_password='*******'
remote_host = '*******'
remote_port='5432'
remote_database = 'ds4a_final'

remote_engine = create_engine(f'postgresql://{remote_username}:{remote_password}@{remote_host}:{remote_port}/{remote_database}',echo=False)

In [3]:
os.chdir(os.path.join('..\..')) #Run only once

In [4]:
basepath = os.getcwd()
ds_path = os.path.join(basepath, 'datasets\\final\\')
pj_path = os.path.join(basepath, 'final\\')

In [5]:
ds_suelos = os.path.join(ds_path, 'processed\\Nei_UsoSuelo.csv')
df_suelos = pd.read_csv(ds_suelos, dtype={'SETU_CCNCT': str})

ds_barrio_dane = os.path.join(ds_path, 'processed\\neighborhood_dane.csv')
df_barrio_dane = pd.read_csv(ds_barrio_dane, dtype={'setu_ccnct': str})

### Create table Neighborhood - DANE

In [6]:
df_barrio_dane['barrio_dane_id'] = df_barrio_dane.index
df_barrio_dane.to_sql(name='barrio_dane', con=local_engine, if_exists = 'replace', index=False)

In [7]:
df_suelos.columns = map(str.lower, df_suelos.columns)
df_suelos.columns = map( lambda s: s.replace(' ', '_').strip(), df_suelos.columns )

In [8]:
dct_barrio_dane = {}
for index, row in df_barrio_dane.iterrows():
    dct_barrio_dane[row['setu_ccnct']] = row['barrio_id']

df_suelos['barrio_id'] = df_suelos['setu_ccnct'].apply(lambda data: (dct_barrio_dane[data] if data in dct_barrio_dane else np.nan))
df_suelos.dropna(inplace=True)
df_suelos['barrio_id'] = df_suelos['barrio_id'].astype(int)

In [9]:
del df_suelos['_']
df_suelos.rename(columns={'unnamed:_0' : 'barrio_suelo_id'}, inplace=True)
del df_suelos['setu_ccnct']

In [10]:
df_suelos.to_sql(name='barrio_suelo', con=local_engine, if_exists = 'replace', index=False)

In [40]:
#Loading all local data
df_barrio = pd.read_sql("SELECT * from barrio", local_engine.connect())
df_barrio_convencion = pd.read_sql("SELECT * from barrio_convencion", local_engine.connect())
df_crime_type= pd.read_sql("SELECT * from crime_type", local_engine.connect())
df_barrio_estrato = pd.read_sql("SELECT * from barrio_estrato", local_engine.connect())
df_barrio_suelo= pd.read_sql("SELECT * from barrio_suelo", local_engine.connect())
df_crime = pd.read_sql("SELECT * from crime", local_engine.connect())

In [ ]:
df_barrio.to_sql(name='barrio', con=remote_engine, if_exists = 'append', index=False)
df_barrio_convencion.to_sql(name='barrio_convencion', con=remote_engine, if_exists = 'append', index=False)
df_crime_type.to_sql(name='crime_type', con=remote_engine, if_exists = 'append', index=False)
df_barrio_estrato.to_sql(name='barrio_estrato', con=remote_engine, if_exists = 'append', index=False)
df_barrio_suelo.to_sql(name='barrio_suelo', con=remote_engine, if_exists = 'append', index=False)
df_crime.to_sql(name='crime', con=remote_engine, if_exists = 'append', index=False)